# 💡 MechOnFluid 本体加载与推理演示手册

> 本文档旨在系统性展示如何基于 MechOn-Fluid 本体进行 **OWL 本体的加载、推理与可视化分析**。  
> 适用于教学演示、模型验证与垂直领域知识工程的入门实践。

你将学习以下内容：
1. 创建，使用聚合文件
2. owl full与owl dl的区别与桥接方法
3. 不同owl本体集间的桥接
4. 推理测试

---

🧑‍💻 文档作者：庞嘉顺  
📅 编写日期：2025 年 7 月  
📘 项目背景：本 notebook 属于 [MechOn] 与 [SessionSync] 项目的基础模块示范之一

本体内容：MechOn-Fluid

<div align="center">
    <img src="../docs/sessync.png" width="700" alt="sessync logo" />
</div>

# 📚 1. 聚合文件的创建与引用（Ontology Aggregation via `owl:imports`）

在本节中，我们将学习如何通过 **聚合文件（aggregate ontology）** 管理多个模块化本体，实现统一引用、自动加载、推理共享等目标。

我们以简化版本的 **MechOn-Fluid** 本体集为例进行演示，说明如何使用 `owl:imports` 构建本体的入口文件。

---

## 🧩 1.1 本体集架构总览

以下是 MechOn-Fluid 项目的模块化结构图，其中每个子模块（如 `mat.ttl`、`phy.ttl`）都是一个独立维护的领域子本体，而 `mechon-fluid.owl` 是顶层聚合文件（也称为 *entrypoint ontology*）。

<div align="center">
    <img src="../docs/core.png" width="700" alt="mechon-fluid structure" />
</div>

---

## 📦 1.2 聚合文件 mechon-fluid.owl 的作用

文件 `mechon-fluid.owl` 作为整个本体系统的统一入口，其核心任务是：

- 使用 `owl:imports` 引用所有子模块；
- 定义统一的命名空间（base URI）；
- 提供版本信息，供推理机和开发工具识别。

一旦加载该文件，所有子本体将自动被导入，无需逐一加载。

---

## 🧰 1.3 聚合文件结构解析

下面是 `mechon-fluid.owl` 的典型结构，采用标准 OWL/RDF Turtle 格式：

```ttl
@prefix :     <https://purl.mechon.org/ont/fluid#> .
@prefix owl:  <http://www.w3.org/2002/07/owl#> .
@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml:  <http://www.w3.org/XML/1998/namespace> .
@prefix xsd:  <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

@base <https://purl.mechon.org/ont/fluid#> .

<https://purl.mechon.org/ont/fluid>
    rdf:type owl:Ontology ;
    owl:versionIRI <https://purl.mechon.org/ont/fluid/0.1.0> ;
    owl:imports 
        <https://purl.mechon.org/ont/calc/0.1.0> ,
        <https://purl.mechon.org/ont/geo/0.1.0> ,
        <https://purl.mechon.org/ont/mat/0.1.0> ,
        <https://purl.mechon.org/ont/phy/0.1.0> .
```
---

上边的owl语句包含三个部分，
### 1. 命名空间与基URI: @prefix / @base
这里的 <https://purl.mechon.org/ont/fluid#> 是该本体的命名空间 IRI，我们可以简称为：

Namespace IRI = 用于构造类、属性、个体等名称的“前缀地址”。

例如：
```ttl
:StressTensor a :TensorField .
```
会被展开为：
```ttl
<https://purl.mechon.org/ont/fluid#StressTensor>
```
---
### 2. 本体资源的标识与元数据定义：rdf:type owl:Ontology
使用 rdf:type owl:Ontology 声明该文件是一个 OWL 本体，并为其指定统一的本体 URI 与版本 URI，便于工具识别和版本控制。
```ttl
<https://purl.mechon.org/ont/fluid>
    rdf:type owl:Ontology ;
    owl:versionIRI <https://purl.mechon.org/ont/fluid/0.1.0> .
```
|字段          |	含义          |
|------------|-------------------|
|rdf:type owl:Ontology	|声明该资源是一个 OWL 本体|
|<.../ont/fluid>	|本体的全局唯一标识符（Ontology IRI）|
|owl:versionIRI	|明确指定当前版本的 URI，有助于版本管理与引用一致性|

> ⚠️  注意：本体 URI（Ontology IRI）与命名空间 URI 并不相同，前者指向“整个本体资源”，后者用于构造实体名称。
---

### 3. 加载子模块 `owl:imports`
聚合文件的关键作用是通过 `owl:imports` 导入其他子模块本体，实现模块化开发、分工维护、统一推理。
```ttl
<https://purl.mechon.org/ont/fluid>
    owl:imports 
        <https://purl.mechon.org/ont/calc/0.1.0> ,
        <https://purl.mechon.org/ont/geo/0.1.0> ,
        <https://purl.mechon.org/ont/mat/0.1.0> ,
        <https://purl.mechon.org/ont/phy/0.1.0> .
```
每一个被导入的模块都应该：

- 是一个合法的 OWL 本体

- 在自身文件中使用 rdf:type owl:Ontology 显式声明

- 定义自己的命名空间与 @base URI

- （可选）使用 owl:versionIRI 声明版本

子模块的本体资源标识示例`mat.core.ttl`
> <https://purl.mechon.org/ont/mat> rdf:type owl:Ontology ;
> 
>                                   owl:versionIRI <https://purl.mechon.org/ont/mat/0.1.0> .


## Practice 1 聚合加载的地址映射问题
在聚合本体的加载中我们还需要有目标本体URI与目标地址的映射。其中目标地址可以在线也可以是本地。

在线获取地址示例：

In [1]:
from rdflib import Graph

# 在线可访问的 OWL 或 TTL 文件（例如发布到 GitHub Pages 或 PURL）
# https://purl.org/mechon/fluid/ontology/geo.core是永久地址，这个地址会将请求转发到下边的url中
url = "https://cocoj-p.github.io/MechOn-fluid/MechOn-fluid/ontology/geo.core.ttl"

g = Graph()
g.parse(url, format="turtle")  # 可省略 format，如果文件结尾是 .ttl 会自动识别

print(f"Loaded {len(g)} triples.")

# （Optional）purl.org是转发地址，rdflib对302Found转发的支持不好，但我们可以使用下边的代码来验证转发状态
# import requests

# url = "https://purl.org/mechon/fluid/ontology/geo.core"
# r = requests.get(url, allow_redirects=True)
# print("Status:", r.status_code)
# print("Redirect chain:", r.history)
# print("Final URL:", r.url)

Loaded 162 triples.


本地地址示例：

In [3]:
from rdflib import Graph

# 
url = "./examples/2/geo.core.ttl"

g = Graph()
g.parse(url, format="turtle")  # 可省略 format，如果文件结尾是 .ttl 会自动识别

print(f"Loaded {len(g)} triples.")


Loaded 162 triples.


由于聚合本体策略需要本体的URI进行链接，所以在单文件的加载之外，我们还需要进行URI与本体地址的映射。

（为了降低算力负担，我们采用本地的简化版本MechOn-Fluid进行测试，所以我们的映射文件全部使用本地地址）

In [5]:
from rdflib import Graph

AggregatedUrl = "./examples/2/mechon-fluid.owl"

# 定义 URI → 本地文件路径的映射关系（模拟 catalog.xml 的功能）
uri_map = {
    "https://purl.mechon.org/ont/calc/0.1.0": "./examples/2/calc.core.ttl",
    "https://purl.mechon.org/ont/mat/0.1.0": "./examples/2/mat.core.ttl",
    "https://purl.mechon.org/ont/phy/0.1.0": "./examples/2/phy.core.ttl",
    "https://purl.mechon.org/ont/geo/0.1.0": "./examples/2/geo.core.ttl",
}
# 加载主图
g = Graph().parse(AggregatedUrl, format="turtle")

# 遍历 owl:imports 并手动加载映射文件
for s, p, o in g.triples((None, g.namespace_manager.expand_curie("owl:imports"), None)):
    uri = str(o)
    if uri in uri_map:
        g.parse(uri_map[uri], format="turtle")
    else:
        print(f"⚠️ 未知 import URI: {uri}，跳过或考虑下载")


让我们来检查是否成功载入聚合文件

In [14]:
import subprocess, tempfile, os
from rdflib import Graph, Namespace
from rdflib.namespace import RDF, RDFS, OWL, URIRef

def check_imports():
    """检查mechon-fluid.owl文件是否成功导入了所有聚合的内容"""
    
    print("=== 检查 mechon-fluid.owl 导入情况 ===\n")
    
    # 1. 检查声明的导入
    print("1. 声明的导入 (owl:imports):")
    ontology_uri = URIRef("https://purl.mechon.org/ont/fluid")
    for s, p, o in g.triples((ontology_uri, OWL.imports, None)):
        print(f"   - {o}")
    
    print("\n2. 检查各个核心文件的内容是否被包含:")
    
    # 2. 检查各个核心文件的内容
    core_files = {
        "calc": "./examples/2/calc.core.ttl",
        "geo": "./examples/2/geo.core.ttl", 
        "mat": "./examples/2/mat.core.ttl",
        "phy": "./examples/2/phy.core.ttl"
    }
    
    for name, file_path in core_files.items():
        print(f"\n   --- {name.upper()} 核心文件检查 ---")
        
        # 加载核心文件
        core_g = Graph().parse(file_path, format="turtle")
        
        # 只统计 URIRef 类型的命名实体
        core_classes = [s for s in core_g.subjects(RDF.type, OWL.Class) if isinstance(s, URIRef)]
        core_obj_props = [s for s in core_g.subjects(RDF.type, OWL.ObjectProperty) if isinstance(s, URIRef)]
        core_data_props = [s for s in core_g.subjects(RDF.type, OWL.DatatypeProperty) if isinstance(s, URIRef)]
        
        print(f"   核心文件包含:")
        print(f"   - 类: {len(core_classes)} 个")
        # for cls in core_classes:
        #     print(f"     · {cls}")
        print(f"   - 对象属性: {len(core_obj_props)} 个")
        # for prop in core_obj_props:
        #     print(f"     · {prop}")
        print(f"   - 数据属性: {len(core_data_props)} 个")
        # for prop in core_data_props:
        #     print(f"     · {prop}")
        
        # 检查主文件中是否包含这些内容
        found_classes = 0
        found_obj_props = 0
        found_data_props = 0
        
        for cls in core_classes:
            if (cls, RDF.type, OWL.Class) in g:
                found_classes += 1
        
        for prop in core_obj_props:
            if (prop, RDF.type, OWL.ObjectProperty) in g:
                found_obj_props += 1
                
        for prop in core_data_props:
            if (prop, RDF.type, OWL.DatatypeProperty) in g:
                found_data_props += 1
        
        print(f"   主文件中找到:")
        print(f"   - 类: {found_classes}/{len(core_classes)} 个")
        print(f"   - 对象属性: {found_obj_props}/{len(core_obj_props)} 个")
        print(f"   - 数据属性: {found_data_props}/{len(core_data_props)} 个")
        
        # 计算导入比例
        if len(core_classes) > 0:
            class_ratio = found_classes / len(core_classes) * 100
            print(f"   - 类导入比例: {class_ratio:.1f}%")
        
        if len(core_obj_props) > 0:
            obj_prop_ratio = found_obj_props / len(core_obj_props) * 100
            print(f"   - 对象属性导入比例: {obj_prop_ratio:.1f}%")
            
        if len(core_data_props) > 0:
            data_prop_ratio = found_data_props / len(core_data_props) * 100
            print(f"   - 数据属性导入比例: {data_prop_ratio:.1f}%")
    
    # 3. 检查主文件的总统计
    print(f"\n3. 主文件 (mechon-fluid.owl) 总统计:")
    main_classes = list(g.subjects(RDF.type, OWL.Class))
    main_obj_props = list(g.subjects(RDF.type, OWL.ObjectProperty))
    main_data_props = list(g.subjects(RDF.type, OWL.DatatypeProperty))
    
    print(f"   - 总类数: {len(main_classes)}")
    print(f"   - 总对象属性数: {len(main_obj_props)}")
    print(f"   - 总数据属性数: {len(main_data_props)}")
    
    # 4. 检查是否有QUDT相关的导入
    print(f"\n4. QUDT相关检查:")
    qudt_entities = [s for s, p, o in g.triples((None, None, None)) 
                    if "qudt.org" in str(s) or "qudt.org" in str(o)]
    print(f"   - QUDT相关实体数: {len(qudt_entities)}")
    
    # 5. 检查命名空间
    print(f"\n5. 命名空间检查:")
    namespaces = g.namespaces()
    for prefix, uri in namespaces:
        print(f"   - {prefix}: {uri}")

if __name__ == "__main__":
    check_imports()

=== 检查 mechon-fluid.owl 导入情况 ===

1. 声明的导入 (owl:imports):
   - https://purl.mechon.org/ont/calc/0.1.0
   - https://purl.mechon.org/ont/geo/0.1.0
   - https://purl.mechon.org/ont/mat/0.1.0
   - https://purl.mechon.org/ont/phy/0.1.0

2. 检查各个核心文件的内容是否被包含:

   --- CALC 核心文件检查 ---
   核心文件包含:
   - 类: 11 个
   - 对象属性: 1 个
   - 数据属性: 0 个
   主文件中找到:
   - 类: 11/11 个
   - 对象属性: 1/1 个
   - 数据属性: 0/0 个
   - 类导入比例: 100.0%
   - 对象属性导入比例: 100.0%

   --- GEO 核心文件检查 ---
   核心文件包含:
   - 类: 29 个
   - 对象属性: 4 个
   - 数据属性: 0 个
   主文件中找到:
   - 类: 29/29 个
   - 对象属性: 4/4 个
   - 数据属性: 0/0 个
   - 类导入比例: 100.0%
   - 对象属性导入比例: 100.0%

   --- MAT 核心文件检查 ---
   核心文件包含:
   - 类: 29 个
   - 对象属性: 2 个
   - 数据属性: 1 个
   主文件中找到:
   - 类: 29/29 个
   - 对象属性: 2/2 个
   - 数据属性: 1/1 个
   - 类导入比例: 100.0%
   - 对象属性导入比例: 100.0%
   - 数据属性导入比例: 100.0%

   --- PHY 核心文件检查 ---
   核心文件包含:
   - 类: 4 个
   - 对象属性: 4 个
   - 数据属性: 2 个
   主文件中找到:
   - 类: 4/4 个
   - 对象属性: 4/4 个
   - 数据属性: 2/2 个
   - 类导入比例: 100.0%
   - 对象属性导入比例: 100.0%
   - 数据属性导入

# 2. OWL Full 与OWL DL的桥接
# 🧠 OWL DL 与 OWL Full 在应用层面的区别

## 🧩 OWL 本体语言的三个子集概览

| 子语言    | 描述                           | 推理支持       | 用例                            |
|-----------|--------------------------------|----------------|---------------------------------|
| OWL Lite  | 精简子集（很少使用）           | 有限推理支持   | 入门级模型                      |
| **OWL DL**   | **描述逻辑兼容，支持完整推理** | ✅ 全面推理      | **科学建模、错误检测、知识一致性验证** |
| OWL Full  | 最自由表达力，无语法限制       | ❌ 无法推理      | 标签本体、视觉可视化、不需推理场景     |

---

## 🚧 应用层面：OWL DL vs OWL Full

| 维度               | OWL DL（描述逻辑）                                     | OWL Full（无约束）                                          |
|--------------------|--------------------------------------------------------|--------------------------------------------------------------|
| **用途定位**         | ✅ 语义推理、错误检测、自动分类                             | 🔄 标签辅助、可视化展示、最大灵活性                                |
| **推理引擎支持**     | ✅ 支持 HermiT, Pellet, FaCT++ 等                        | ❌ 无法使用标准推理机，因语义不明                                 |
| **语法限制**         | ❗ 有限制（类/属性/个体分层、不能将类作为个体）               | ✅ 无限制（类可以是个体，属性可以是类）                            |
| **一致性检查能力**   | ✅ 支持类型冲突检测、单位约束推断、枚举范围校验等                  | ❌ 不可做一致性推理，错误不能暴露                                  |
| **与 SHACL/规则集成**| ✅ 可与 SHACL、SWRL 配合，形成规范数据流                        | ⚠️ 只能做结构检查，缺乏语义可验证性                                 |
| **适合谁用？**       | 💼 科研、工程建模、知识验证、规则约束、语义纠错                    | 🎨 可视化图谱、社交图谱、注释系统、非形式化本体                         |

---

## ✅ 为何你需要 OWL DL（以启用 HermiT 推理与报错）

你当前目标是：

- 使用 **推理机 HermiT** 执行 `owl:imports` 后的一致性验证；
- 检测如：
  - 错误的单位匹配（`hasUnit` 与不兼容类）；
  - 属性值不在枚举范围；
  - 类之间的 `disjointWith` 冲突；
- 对大型科学模型（如流体力学本体、张量模型）进行严格的语义检查与自动补全。

🔍 这些 **只能在 OWL DL 模型下实现**，因为：
- HermiT 推理器仅支持 OWL DL；
- OWL DL 保证语义明确、可判定；
- 如果你的文件是 OWL Full（比如混用 `owl:Class` 和个体），推理机会直接拒绝解析，无法报错，也无法检查错误。

---

## 📌 常见触发 OWL Full 的设计陷阱（请避免）
在 OWL 本体开发中，一些常见的建模方式**表面上合法，但在 OWL DL 中会导致语义混乱，从而进入 OWL Full 模式**，推理机（如 HermiT）将拒绝执行。

以下是典型陷阱及建议替代方式：

| ⚠️ 误用行为 | ❌ 会导致 OWL Full | ✅ 推荐做法（OWL DL 合法） |
|-------------|---------------------|------------------------------|
| 同一个标识符既用作类又作为个体 | ❗ 类-个体角色混淆 | 使用 **punning** 或显式拆分为两个 IRI |
| `rdf:type owl:Class` 和 `rdf:type :ClassA` 同时存在 | ❗ ABox/TBox 混用 | 用注释说明，不将类本身作为个体处理 |
| `owl:Class` 出现在属性值中 | ❗ 类作为值是不允许的 | 使用代理对象或 `rdfs:seeAlso` |
| 使用 `owl:Ontology` 参与推理三元组 | ❗ 本体本体间混用 | 将其作为文档元数据，不纳入推理域 |
| 类直接作为 `owl:hasValue` 的值 | ❗ 类参与了 ABox 声明 | 用命名个体替代或转为 `someValuesFrom` |

---

### 🧠 重点：什么是 Punning？（名称复用机制）

**Punning 是 OWL DL 中的合法机制，用于让“同一个标识符”在不同上下文中扮演不同语义角色**，而不触发 OWL Full。

#### ✅ 示例：合法的 punning 用法

```ttl
ex:Temperature a owl:Class .      # 用作一个概念类
ex:Temperature a ex:PhysicalType . # 同时也是一个个体
```
在 OWL DL 中，这不会引起错误，因为：

ex:Temperature 在 owl:Class 的上下文中是一个类

ex:Temperature 在 rdf:type ex:PhysicalType 中是一个个体

这种语义分离称为 **punning（词义歧义）**。
---

### ⚠️ Punning 的注意事项

- 不是所有推理引擎都支持 punning（但 HermiT ✅ 支持）

- 尽管合法，不建议滥用，特别是在教学、可维护性和导出语义场景中

- 若使用 punning，请明确说明该标识符在文档中扮演的多个角色

---

### ✅ 实践建议：如何安全使用 Punning

用于表示 单位、类型标签、数学定义等抽象值，如：
```ttl
:Pascal a owl:NamedIndividual, :PressureUnit .
```
避免让 owl:Class 的类直接参与其他类定义关系，如：

❌ 不推荐：

```ttl
:FlowEquation owl:equivalentClass :Equation .
```
✅ 推荐：

```ttl
:FlowEquation owl:equivalentClass [
    a owl:Class ;
    owl:intersectionOf (:Equation :IncompressibleFlow)
] .
```
在多角色情境中尽可能明确注释说明，如：

```ttl
# ex:Temperature used both as class and instance (punning)
```
---

## ✅ 建议实践：确保 OWL DL 的建模与推理兼容性

1. **使用独立命名空间定义不同角色（类、属性、个体）**
2. **使用 Protégé 的 DL Profile 校验工具**
3. **用 HermiT + rdflib 组合做一致性检查**
4. **导出 `.ttl` 文件时注意不要混入 OWL Full 特性**
5. **避免使用 `owl:Class` 作为 ABox 个体参与关系**

---

## 📎 总结

- ✅ OWL DL 是“语义网中的编程语言”，支持形式逻辑推理与语义验证
- ❌ OWL Full 是“语义网中的自然语言”，用于描述而非验证
- 💡 如果你希望用 HermiT 报错、检查单位冲突、分类错误、推理出缺省关系，**请务必使用 OWL DL 并保持规范建模**

---


## 2.1 OWL Full⇄DL 的最佳实践 Bridging

在构建 OWL DL 本体时，我们常常需要引入第三方已有本体资源以增强可重用性与互操作性，例如 QUDT、Schema.org、FOAF 等。**然而，这些外部本体往往属于 OWL Full，而我们的项目要求保持在 OWL DL 范式中以支持完整的逻辑推理（如使用 HermiT 报错和验证）**。

这就提出了一个常见挑战：  
> **如何在不修改第三方 OWL Full 本体的前提下，将其桥接进我们自身的 OWL DL 推理环境？**

我们称这种策略为 **OWL Full⇄DL 的桥接（bridging）实践**。

---

### 🎯 为什么要桥接而不是直接修改？

- ✅ **第三方本体受他人维护**，我们无权也不应修改其原始结构；
- ✅ **遵循语义网开放性精神**，保持 URI 可复用与语义对接；
- ✅ **原始本体可能包含 OWL Full 特性**（如 punning、类即个体）；
- ✅ **我们希望启用 OWL DL 推理能力**（如一致性校验、错误定位、类型补全）；
- ✅ **通过桥接层定义 DL 合法的等价结构**，避免直接引入 OWL Full 部分破坏推理链。

---

### ✅ 桥接方法（Bridging Strategy）

我们的做法是：

1. **单独建立一个 `*-bridge.ttl` 文件（或模块）**，用于定义 OWL DL 合法的别名、限制、类型约束；
2. 使用 `owl:equivalentClass`、`owl:equivalentProperty` 等在 **DL 合法语境中重建引用**；
3. 将原始 OWL Full 本体以只读方式引入（例如 `imports` 或 SPARQL `FROM NAMED`）；
4. 所有面向推理和验证的操作都只在 **bridge 层和内部 DL 本体上进行**。

---

### 🧠 示例（以 QUDT 单位为例）

```ttl
# QUDT 是 OWL Full 本体，直接引入会破坏 DL
qudtunit:PA  a owl:NamedIndividual .  # 原始定义

# 我们在 bridge 中重写
:Pascal a owl:Class ;
    owl:equivalentClass [
        a owl:Restriction ;
        owl:onProperty :hasUnit ;
        owl:hasValue qudtunit:PA
    ] .
```

---
## ✅ 好处
- ✅ 不改变原始本体，保持可更新性与一致性；

- ✅ 保持我们内部模型的 OWL DL 合法性；

- ✅ 可以用推理机对 bridge+core 本体集进行严谨校验；

- ✅ 保证语义对齐，便于协同开发和扩展。

## 📎 实践建议
- 为每一个外部 OWL Full 模块单独建一个对应的 *-bridge.ttl；

- 使用清晰命名空间区分 bridge vs imported；

- 定期用 HermiT 对 bridge+DL 主本体进行一致性验证；

- 避免将 OWL Full 直接 merge 到主本体中，防止污染推理环境。


---

## Practice 2 Bridging

<div align="center">
    <img src="../docs/bridging.png" width="900" alt="sessync logo" />
</div>

我们采用如图所示的“桥接对齐”模式，将外部 OWL Full 本体（如 QUDT）整合进 OWL DL 环境中。中间层由两个部分构成：

- **Bridge Layer**：从 QUDT 中抽取所需实体，进行语义扁平化，构成 DL-compatible 的简化视图；
- **Alignment Layer**：以三元组的形式，将我们的领域概念（如 mat:Density）与 bridge 中的 QUDT 概念（如 qudtqk:MassDensity）建立语义对应。

该结构使得我们在不修改原始 QUDT 的情况下，实现了开放语义互操作性与 OWL DL 推理能力的兼容共存。
